### 연습
- data 폴더 안의 가전 폴더의 모든 json 파일을 하나의 데이터프레임으로 단순 행 결합
- 데이터에서 결측치를 확인
    - GeneralPolaruty 컬럼에서 결측치 발견
- 결측치가 포함된 데이터를 따로 저장 (na_df)
- 결측치 제거
- 'RawText', 'GeneralPolarity' 컬럼을 제외한 나머지 컬럼 제거
- 'GeneralPolarity' 컬럼의 이름을 'labels'로 변경
- RawText는 텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제외, 문자열 앞 뒤 공백 제거)
- labels 데이터에서 -1과 0은 0으로 1은 1로 데이터를 변경 -> 해당 컬럼의 dtype을 int - 변경 -> BERTmodel에서 선형 모델로 확률을 예측하기때문에 labels가 위치 값
- train, test의 형태로 데이터를 9:1의 비율로 나눠준다.
    - labels를 기준으로 계층화 분할
- 데이터프레임을 Dataset의 형태로 변환
- token화 작업은 AutoTokenizer를 이용하여 skt/kobert-base-v1 모델을 이용하여 토큰화
- 같은 모델을 로드하여 BertModel + Linear 모델 정의
- Trainer, TrainingArguments를 이용하여 학습 관리 객체 생성
- 학습 -> na_df에서 상위 5개의 RawText을 확인하여 예측

In [44]:
import os       # 파일 목록 생성
from glob import glob
import pandas as pd
import re
import torch
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, BertModel, Trainer, TrainingArguments
import numpy as np

---
파일 목록 확인

In [23]:
# 방법 1. os 라이브러리를 이용
file_path = '../data/가전/'
file_list = os.listdir(file_path)
pd.read_json(file_path + file_list[0])

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspects
0,112038,엄마가 갑자기 전화하시더니 집에서 사용하는 노래방 마이크가 사고 싶다고 하시네요.....,SNS,가전,영상/음향가전,(1+1세트) TJ 태진 블루투스 마이크 / 무선 노래방 마이크,3,323,73,20221110,0.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '우리나라 ..."
1,114572,누가 사용하다가 반품한 것만 같은 제품이 와서 조금 당황스러웠어요ㅠㅠ일단 겉 부분에...,SNS,가전,영상/음향가전,(1등급)삼성 QLED 4K TV 138cm(55형) KQ55QT67AFXKR+삼성...,1,317,70,20221119,-1.0,"[{'Aspect': '품질', 'SentimentText': '겉 부분에 여기저기..."
2,114573,노트북과 TV를 연결해서 모니터 하나에서 다 보려고 구매했어요.여러 제품과 비교하고...,SNS,가전,영상/음향가전,4K HDMI 2.0 양방향 선택기,1,311,70,20221113,0.0,"[{'Aspect': '기능', 'SentimentText': ' 그래도 괜찮은 건..."
3,114574,너무 별로예요 ㅡㅡ… 이 정도 퀄리티인 줄 알았으면 안 샀을 거 같네요… 음질이 거...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,306,84,20221121,-1.0,"[{'Aspect': '품질', 'SentimentText': '너무 별로예요 ㅡㅡ..."
4,114575,소음이 섞여서 나는 편이에요... 사이즈도 작고 휴대하기 간편해서 손이 자주 가긴 ...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,303,73,20221121,-1.0,"[{'Aspect': '소음', 'SentimentText': '소음이 섞여서 나는..."
...,...,...,...,...,...,...,...,...,...,...,...,...
96,114667,이전부터 구매하고 싶어서 계속 눈여겨보다 구매한 락클래식이에요. 포장을 제거하고 제...,SNS,가전,영상/음향가전,엠지텍 락클래식Q9900 (정품),1,290,61,20221120,-1.0,"[{'Aspect': '품질', 'SentimentText': '마감은 좀 문제가 ..."
97,114668,요즘 집에서 작업하면서 핸드폰으로 음악을 들으니 전화를 하거나 핸드폰을 이용할때 자...,SNS,가전,영상/음향가전,오아 아이브릭 휴대용 블루투스 미니 스피커,1,338,78,20221110,-1.0,"[{'Aspect': '디자인', 'SentimentText': '디자인이 좀 그렇..."
98,114669,"처음 들어보는 생소한 브랜드의 tv라 걱정하면서 구입했는데, 역시나 후회 중입니다....",SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,333,79,20221116,-1.0,"[{'Aspect': '소음', 'SentimentText': '별로 소리를 키우지..."
99,114670,최신 기종이라고 해서 기대했는데 구기종보다 못하네요. 소재가 별로여서 예쁜 디자인이...,SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,324,74,20221116,-1.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '최신 기종..."


In [24]:
# 방법 2. glob 라이브러리의 glob을 이용
file_list2 = glob("../data/가전/*.json")
pd.read_json(file_list2[0])

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspects
0,112038,엄마가 갑자기 전화하시더니 집에서 사용하는 노래방 마이크가 사고 싶다고 하시네요.....,SNS,가전,영상/음향가전,(1+1세트) TJ 태진 블루투스 마이크 / 무선 노래방 마이크,3,323,73,20221110,0.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '우리나라 ..."
1,114572,누가 사용하다가 반품한 것만 같은 제품이 와서 조금 당황스러웠어요ㅠㅠ일단 겉 부분에...,SNS,가전,영상/음향가전,(1등급)삼성 QLED 4K TV 138cm(55형) KQ55QT67AFXKR+삼성...,1,317,70,20221119,-1.0,"[{'Aspect': '품질', 'SentimentText': '겉 부분에 여기저기..."
2,114573,노트북과 TV를 연결해서 모니터 하나에서 다 보려고 구매했어요.여러 제품과 비교하고...,SNS,가전,영상/음향가전,4K HDMI 2.0 양방향 선택기,1,311,70,20221113,0.0,"[{'Aspect': '기능', 'SentimentText': ' 그래도 괜찮은 건..."
3,114574,너무 별로예요 ㅡㅡ… 이 정도 퀄리티인 줄 알았으면 안 샀을 거 같네요… 음질이 거...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,306,84,20221121,-1.0,"[{'Aspect': '품질', 'SentimentText': '너무 별로예요 ㅡㅡ..."
4,114575,소음이 섞여서 나는 편이에요... 사이즈도 작고 휴대하기 간편해서 손이 자주 가긴 ...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,303,73,20221121,-1.0,"[{'Aspect': '소음', 'SentimentText': '소음이 섞여서 나는..."
...,...,...,...,...,...,...,...,...,...,...,...,...
96,114667,이전부터 구매하고 싶어서 계속 눈여겨보다 구매한 락클래식이에요. 포장을 제거하고 제...,SNS,가전,영상/음향가전,엠지텍 락클래식Q9900 (정품),1,290,61,20221120,-1.0,"[{'Aspect': '품질', 'SentimentText': '마감은 좀 문제가 ..."
97,114668,요즘 집에서 작업하면서 핸드폰으로 음악을 들으니 전화를 하거나 핸드폰을 이용할때 자...,SNS,가전,영상/음향가전,오아 아이브릭 휴대용 블루투스 미니 스피커,1,338,78,20221110,-1.0,"[{'Aspect': '디자인', 'SentimentText': '디자인이 좀 그렇..."
98,114669,"처음 들어보는 생소한 브랜드의 tv라 걱정하면서 구입했는데, 역시나 후회 중입니다....",SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,333,79,20221116,-1.0,"[{'Aspect': '소음', 'SentimentText': '별로 소리를 키우지..."
99,114670,최신 기종이라고 해서 기대했는데 구기종보다 못하네요. 소재가 별로여서 예쁜 디자인이...,SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,324,74,20221116,-1.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '최신 기종..."


---
모든 데이터프레임을 로드하여 하나의 데이터프레임으로 생성

In [25]:
# 데이터프레임을 누적하여 결합할 빈 데이터프레임 생성
df = pd.DataFrame()

# 방법 1.
# file_list2 만큼 반복 실행하는 반복문을 생성
for file_name in file_list2:
    data = pd.read_json(file_name)
    df = pd.concat([df, data], axis= 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [26]:
# 방법 2.
pd.concat(
    [ pd.read_json(file_name) for file_name in file_list2 ]
).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [27]:
# 방법 3.
# python 기본 내장 map() 함수 사용 - map( function, list )
# 강사님이 map() 사용에 익숙해지게 하기 위해 급조해보신 것
pd.concat( list(map( lambda file_name: pd.read_json(file_name), file_list2)) ).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


---
결측치 처리

In [28]:
# 결측치 개수 확인
df['GeneralPolarity'].isna().sum()

np.int64(378)

In [29]:
# 결측치인 데이터를 
na_df = df.loc[
    df['GeneralPolarity'].isna(),
]
na_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, 13 to 88
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            378 non-null    int64  
 1   RawText          378 non-null    object 
 2   Source           378 non-null    object 
 3   Domain           378 non-null    object 
 4   MainCategory     378 non-null    object 
 5   ProductName      378 non-null    object 
 6   ReviewScore      378 non-null    int64  
 7   Syllable         378 non-null    int64  
 8   Word             378 non-null    int64  
 9   RDate            378 non-null    int64  
 10  GeneralPolarity  0 non-null      float64
 11  Aspects          378 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 38.4+ KB


In [30]:
# df에서 결측치를 제거
df.dropna(inplace= True)

---
컬럼 처리

In [31]:
# 특정 컬럼 두 개만 선택
df = df[ ['RawText', 'GeneralPolarity'] ]

In [32]:
# 특정 컬럼의 이름 변경
# rename() 함수 사용
df.rename(columns={
    'GeneralPolarity': 'labels'
}, inplace= True)

In [33]:
df['labels'].value_counts()
# 긍정(1)-무난(0)-부정(-1)의 데이터가 불균형하고, dtype은 int이지만 float 형태임

labels
 1.0    2220
 0.0     944
-1.0     514
Name: count, dtype: int64

- RawText -> 정규화
- labels -> 타입을 int로, -1을 0으로 변경

In [34]:
# labels 컬럼의 데이터 타입을 int로 고정시킨다.
df['labels'] = df['labels'].astype(int)

In [ ]:
# 정규화
def normalize(x):
    # x: apply() 함수를 이용하여 들어오는 Series 형태의 데이터
    x['RawText'] = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", x['RawText'])
    x['RawText'] = re.sub(r"\s+", " ", x['RawText'])

    # 이진 분류를 위해 -1을 0으로 수정
    # if x['labels'] == -1:
    #     x['labels'] = 0

    # 삼중 분류하려면?
    # 단계 1. 위 if문 빼기
    #   ⇒ IndexError: Target -1 is out of bounds

    # 단계 2. labels가 -1, 0, 1로 이루어져 예측 확률에서 -1의 위치가 존재하지 않아 Error가 발생했으니, labels 데이터에 1을 증가시킴.
    # x['labels'] += 1
    #   ⇒ IndexError: Target 2 is out of bounds

    # 단계 3. 예측 확률에서 3의 위치가 존재하지 않아 Error가 발생했으니, 
    # Trainer의 설정계수인 args나, 예측값을 만들어주는 model 중 원인이 있을텐데, args를 살펴보니 pred의 길이와 연관된 매개변수가 없으니 model을 살펴본다.
    # BERTClsHead 에서 '결과의 피쳐 개수'를 결정하는 'num_label'의 값을 2로 설정해뒀음 --> 원인?!
    # model = BERTClsHead(MODEL_NAME, num_label=2) -> model = BERTClsHead(MODEL_NAME, num_label=3) 으로 수정해봄.
    #   ⇒ ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']

    # 단계 4. metrics 함수의 f1_score에 매개변수 average= 'macro' 추가
    #   ⇒ 정상적으로 실행되지만, 성능은 떨어지게 됨.

    # 단계 5. 실제 데이터로 평가할 때의 출력 결과를 부정, 긍정 -> 부정, 중립, 긍정으로 수정
    
    return x

df = df.apply(normalize, axis= 1)
df

# df.apply(lambda x: print(x), axis= 1)
# 컬럼을 기준으로 데이터를 나눠서 보여준다.
# axis = 0
    # x에 df['RawText'] 대입되어 한 번 실행
    # x에 df['labels'] 대입되어 한 번 실행
# axis = 1
    # x에 df.iloc[0, ] 대입되어 한 번 실행
    # x에 df.iloc[1, ] 대입되어 한 번 실행

,RawText,labels
0,엄마가 갑자기 전화하시더니 집에서 사용하는 노래방 마이크가 사고 싶다고 하시네요.....,1
1,누가 사용하다가 반품한 것만 같은 제품이 와서 조금 당황스러웠어요 일단 겉 부분에 ...,1
2,노트북과 TV를 연결해서 모니터 하나에서 다 보려고 구매했어요.여러 제품과 비교하고...,1
3,너무 별로예요 이 정도 퀄리티인 줄 알았으면 안 샀을 거 같네요 음질이 거의 입 안...,1
4,소음이 섞여서 나는 편이에요... 사이즈도 작고 휴대하기 간편해서 손이 자주 가긴 ...,1
...,...,...
95,사진으로는 잘 체감하지 못했는데 실제 실물을 보니 디자인이 조금 충격적이라고나 할까...,2
96,이 에어컨의 제일 큰 장점은 조작법에 있는 것 같아요.인공지능 조작이 가능해서 한번...,2
97,이 제품을 추천하는 이유는요 일단 LED를 통해 공기질을 눈으로 확인할 수 있고 터...,2
98,인터넷에서 구매하였는데요 이 공기청정기 처음 발견하고 처음에는 오잉 이게뭐지 했어요...,2


Target(종속)인 'labels' 데이터는 -1, 0, 1로 이루어져 있다.</br>
Bert 모델, Linear 모델에서 예측값은 `pred = [ 0.3, 0.4, 0.3 ]`</br>
이 중 가장 큰 확률의 index가 target이라는 뜻.</br>
리스트인 pred의 index는 0, 1, 2인데 인덱스값이 -1인 index를 찾으려 한 것.</br>
index가 -1 이라는 것은 list(len(list) -1) 를 의미함.</br>
따라서 알맞은 값을 찾을 수 없어 Error 발생.</br>
-> labels 데이터나 pred 데이터 중 하나를 바꿔야 하는데,</br>
- labels는 처음에 로드한 데이터프레임에 위치.
- pred는 사용하는 model의 결과값.
-> labels 데이터를 바꾸는 게 편리함.</br>
=> 정규화 함수를 수정

In [83]:
df['labels'].value_counts()

labels
2    2220
1    1458
Name: count, dtype: int64

---
데이터셋 분할

In [84]:
# Train, Test 데이터셋으로 분할 - labes의 데이터로 계층화
train_df, test_df = train_test_split(
    df, test_size=0.1, random_state=42, stratify=df['labels']
)
len(train_df)

3310

In [85]:
# DataFrame -> Dataset 형태로 파싱
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

---
토큰화

In [86]:
# Tokenizer
MODEL_NAME = 'skt/kobert-base-v1'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

In [87]:
def tok_fn(batch):
    # batch -> Dataset에서 map() 함수로 데이터를 특정 개수로 묶어 보낸다. (기본값)
    result = tokenizer(batch['RawText'], truncation=True, max_length=128)
    # 토큰 개수에 제한을 두고, 해당 개수보다 길다면 잘라준다.
    return result

train_tok = train_ds.map(tok_fn, batched=True, remove_columns=['RawText'])
test_tok = test_ds.map(tok_fn, batched=True, remove_columns=['RawText'])

Map: 100%|██████████| 368/368 [00:00<00:00, 1149.20 examples/s]


In [88]:
# BERT 모델 + Linear 모델 정의
class BERTClsHead(nn.Module):
    def __init__(self, model_name, num_label = 2, dropout = 0.1):
        super().__init__()
        # 기존의 학습된 모델 로드
        self.backbone = BertModel.from_pretrained(model_name)
        # 로드한 모델에서 출력 차원의 개수를 저장
        hidden = self.backbone.config.hidden_size

        # 일정 데이터를 소실시키는 dropout
        self.dropout = nn.Dropout(dropout)

        # nn.Linear(입력, 출력)
        self.classifier = nn.Linear(hidden, num_label)

        # 작업의 안정성을 위해, tokenizer에서 사용하는 패딩 토큰의 id를 모델의 패딩 id에 대입
        self.backbone.config.pad_token_id = tokenizer.pad_token_id
    
    # 순전파 함수
    # **kwargs: 값들을 다른 형태로도 받을 수 있게 함
    def forward(self, input_ids = None, attention_mask = None, labels = None, **kwargs):
        # 출력
        out = self.backbone(input_ids= input_ids, attention_mask= attention_mask)
        
        pooled = out.last_hidden_state[:, 0]

        logits = self.classifier( self.dropout(pooled) )

        result = {"Logits": logits}

        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            result['loss'] = loss
        return result


In [94]:
# 모델을 생성
model = BERTClsHead(MODEL_NAME, num_label=3)
# num_label = 2 --> 이진 분류
# num_label = 3 --> 삼중 분류

In [99]:
# 평가에서 사용할 함수 정의
def metrics(eval_pred):
    logits, y = eval_pred
    pred = logits.argmax(-1)
    return {
        'accuracy_score': accuracy_score(y, pred),
        'f1_score': f1_score(y, pred, average='macro')
    }

In [100]:
# Trainer가 사용할 설정 계수를 정의
args = TrainingArguments(
    output_dir= './kobert_from_bertmodel',
    eval_strategy= 'epoch',
    save_strategy= 'epoch',
    num_train_epochs= 2,            # 성능에 영향   # 일반적으로 3-5회
    learning_rate= 5e-5,            # 성능에 영향   # 일반적으로 5e-5, 4e-5, 3e-5
    weight_decay= 0.01,             # 성능에 영향
    warmup_ratio= 0.01,             # 성능에 영향
    logging_steps= 50,
    load_best_model_at_end= True,
    metric_for_best_model= 'f1',
    greater_is_better= True,
    report_to= []                   # 비활성화
)

In [101]:
trainer = Trainer(
    model= model,   # 모델
    args= args,     # 설정값들
    train_dataset= train_tok,
    eval_dataset= test_tok,
    tokenizer= tokenizer,
    compute_metrics= metrics
)

C:\Users\student\AppData\Local\Temp\ipykernel_16280\4120448850.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [102]:
eval_res = trainer.evaluate()

print(eval_res)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.0921717882156372, 'eval_model_preparation_time': 0.0049, 'eval_accuracy_score': 0.3451086956521739, 'eval_f1_score': 0.24613992643732419, 'eval_runtime': 58.7526, 'eval_samples_per_second': 6.264, 'eval_steps_per_second': 0.783}


In [103]:
# 새로운 문장에 대해서 평가
samples = na_df['RawText'].head().to_list()

In [104]:
enc = tokenizer(
    samples,
    return_tensors = 'pt',
    padding= True,
    truncation= True,
    max_length= 128
)

In [105]:
with torch.no_grad():
    out = model(**enc)
    probs = torch.softmax(out['Logits'], dim= -1)

In [ ]:
# 이중 분류할 때
# for s, p in zip(samples, probs):
#     print(f"""
#     리뷰 - "{s[:50]}"
#       ( 부정: {p[0] :.3f}    긍정: {p[1] :.3f}    예측: {p.argmax()} )""")


    리뷰 - "귀에서 자꾸 빠져요.귀에 꼽는재 질이 미끄러운 재질이라 작은 소품이지만 재질만 바꾸어도 안"
      ( 부정: 0.299    긍정: 0.367    예측: 1 )

    리뷰 - "아이를 출산한 기념으로 TV를 바꿨습니다. 그전에 쓰던 TV가 꽤나 무거워서 떨어지거나 하"
      ( 부정: 0.331    긍정: 0.342    예측: 1 )

    리뷰 - "화면에 노이즈가 생깁니다. 저희 가족 중에 아무도 TV 화면을 건드리거나 하지 않았는데 사"
      ( 부정: 0.318    긍정: 0.388    예측: 1 )

    리뷰 - "이번에 이사하면서 우리 따님께서 방에 TV가 있으면 좋겠다고 하여, 방에서 사용할 TV로 "
      ( 부정: 0.325    긍정: 0.341    예측: 1 )

    리뷰 - "기존에 사용하던 무선이어폰이 오래되어서 배터리가 광탈하는 바람에 새로운 상품이 필요해서 급"
      ( 부정: 0.314    긍정: 0.374    예측: 1 )


In [107]:
# 삼중 분류할 때
for s, p in zip(samples, probs):
    print(f"""
    리뷰 - "{s[:50]}"
      ( 부정: {p[0] :.3f}    중립: {p[1] :.3f}    긍정: {p[2] :.3f}    예측: {p.argmax()} )""")


    리뷰 - "귀에서 자꾸 빠져요.귀에 꼽는재 질이 미끄러운 재질이라 작은 소품이지만 재질만 바꾸어도 안"
      ( 부정: 0.299    중립: 0.367    긍정: 0.334    예측: 1 )

    리뷰 - "아이를 출산한 기념으로 TV를 바꿨습니다. 그전에 쓰던 TV가 꽤나 무거워서 떨어지거나 하"
      ( 부정: 0.331    중립: 0.342    긍정: 0.327    예측: 1 )

    리뷰 - "화면에 노이즈가 생깁니다. 저희 가족 중에 아무도 TV 화면을 건드리거나 하지 않았는데 사"
      ( 부정: 0.318    중립: 0.388    긍정: 0.294    예측: 1 )

    리뷰 - "이번에 이사하면서 우리 따님께서 방에 TV가 있으면 좋겠다고 하여, 방에서 사용할 TV로 "
      ( 부정: 0.325    중립: 0.341    긍정: 0.334    예측: 1 )

    리뷰 - "기존에 사용하던 무선이어폰이 오래되어서 배터리가 광탈하는 바람에 새로운 상품이 필요해서 급"
      ( 부정: 0.314    중립: 0.374    긍정: 0.312    예측: 1 )
